# House Price Prediction
- 출처 : https://www.kaggle.com/competitions/2019-2nd-ml-month-with-kakr

- 집의 정보를 이용하여 가격을 예측하는 방식 (Regression 문제)
- RMSE를 이용하여 모델의 정확도를 평가함

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

# import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb 

import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기

In [2]:
data_dir = 'data/housing'

train_data_path = join(data_dir, 'train.csv')
test_data_path = join(data_dir, 'test.csv') 

train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)

In [3]:
print(train.shape, test.shape)
train.head()

(15035, 21) (6468, 20)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


- 15035의 train 데이터와 6468개의 test 데이터셋으로 이루어져있다

## EDA

### 데이터 살펴보기

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15035 entries, 0 to 15034
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15035 non-null  int64  
 1   date           15035 non-null  object 
 2   price          15035 non-null  float64
 3   bedrooms       15035 non-null  int64  
 4   bathrooms      15035 non-null  float64
 5   sqft_living    15035 non-null  int64  
 6   sqft_lot       15035 non-null  int64  
 7   floors         15035 non-null  float64
 8   waterfront     15035 non-null  int64  
 9   view           15035 non-null  int64  
 10  condition      15035 non-null  int64  
 11  grade          15035 non-null  int64  
 12  sqft_above     15035 non-null  int64  
 13  sqft_basement  15035 non-null  int64  
 14  yr_built       15035 non-null  int64  
 15  yr_renovated   15035 non-null  int64  
 16  zipcode        15035 non-null  int64  
 17  lat            15035 non-null  float64
 18  long  

    ID : 집을 구분하는 번호
    date : 집을 구매한 날짜
    price : 타겟 변수인 집의 가격
    bedrooms : 침실의 수
    bathrooms : 침실당 화장실 개수
    sqft_living : 주거 공간의 평방 피트
    sqft_lot : 부지의 평방 피트
    floors : 집의 층 수
    waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
    view : 집이 얼마나 좋아 보이는지의 정도
    condition : 집의 전반적인 상태
    grade : King County grading 시스템 기준으로 매긴 집의 등급
    sqft_above : 지하실을 제외한 평방 피트
    sqft_basement : 지하실의 평방 피트
    yr_built : 집을 지은 년도
    yr_renovated : 집을 재건축한 년도
    zipcode : 우편번호
    lat : 위도
    long : 경도
    sqft_living15 : 2015년 기준 주거 공간의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)
    sqft_lot15 : 2015년 기준 부지의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)

- 문자열 type의 데이터는 없고 모두 수치형 데이터로 구성되어있다.
- date가 object 타입이긴한데 데이터 특성상 시계열 데이터이므로 datetime 형태로 변환해 줄 것이다
- 수치형 데이터이더라도 이진분류가 되는 binary 형태나 등급같은 것은 범주형 데이터의 형태일 것으로 보인다.
    - 수치형과 범주형 데이터를 나눠서 분석을 해줄 것이다

## ID : 집을 구분하는 번호

In [5]:
train.shape

(15035, 21)

In [6]:
train.id.nunique()

15035

- id의 값은 unique 하므로 인덱스와 일치하기에 id를 index로 변경하거나 없애주어도 된다
- test data도 동일하게 전처리를 진행한다

In [7]:
# id를 index로
# train.set_index('id', inplace=True)
# test.set_index('id', inplace=True)

In [8]:
del train['id']
del test['id']

In [9]:
train.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [10]:
test.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
1,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
2,20140512T000000,4,4.50,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
3,20150415T000000,3,1.00,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
4,20150312T000000,3,2.50,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


## date : 집을 구매한 날짜

- T 뒤에는 의미없는 숫자들인 것 같고 앞에는 날짜정보를 가지고 있으므로 전처리를 통해 datetime형식으로 변환해준다

In [11]:
def extract_date_info(train):
    # 날짜 정보 분리 후 datetime으로 형식 변환
    train['date'] = train['date'].str.split('T').str[0]
    train['date'] = pd.to_datetime(train['date'])

    # 시간 관련 여러 파생변수 추출
    train['year'] = train.date.dt.year
    train['month'] = train.date.dt.month
    train['day'] = train.date.dt.day

    train['day_of_week'] = train.date.dt.day_of_week
    train['day_name'] = train.date.dt.day_name()
    train['quarter'] = train.date.dt.quarter
    
    # drop 하지 않고 시간에 따른 가격 변화를 보기 위해 남겨두자
    train['date'] = train['date'].astype('int')

In [12]:
extract_date_info(train)

In [13]:
train.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,lat,long,sqft_living15,sqft_lot15,year,month,day,day_of_week,day_name,quarter
0,1413158400000000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,47.5112,-122.257,1340,5650,2014,10,13,0,Monday,4
1,1424822400000000000,180000.0,2,1.00,770,10000,1.0,0,0,3,...,47.7379,-122.233,2720,8062,2015,2,25,2,Wednesday,1
2,1424217600000000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,47.6168,-122.045,1800,7503,2015,2,18,2,Wednesday,1
3,1403827200000000000,257500.0,3,2.25,1715,6819,2.0,0,0,3,...,47.3097,-122.327,2238,6819,2014,6,27,4,Friday,2
4,1421280000000000000,291850.0,3,1.50,1060,9711,1.0,0,0,3,...,47.4095,-122.315,1650,9711,2015,1,15,3,Thursday,1


In [14]:
# test data에도 마찬가지로 같이 전처리를 해준다
extract_date_info(test)

### 시간 관련 변수 시각화

In [15]:
date_col = ['year','month','day','day_of_week','day_name','quarter']

In [16]:
train[date_col]

,year,month,day,day_of_week,day_name,quarter
0,2014,10,13,0,Monday,4
1,2015,2,25,2,Wednesday,1
2,2015,2,18,2,Wednesday,1
3,2014,6,27,4,Friday,2
4,2015,1,15,3,Thursday,1
...,...,...,...,...,...,...
15030,2014,10,14,1,Tuesday,4
15031,2015,3,26,3,Thursday,1
15032,2014,5,21,2,Wednesday,2
15033,2015,2,23,0,Monday,1


- 2014년에 비해 2015년에 집 구매가 줄었다
- 5, 6, 7월에 주로 집을 구매하며 1월, 2월, 11월, 12월의 구매가 현저히 적다
- 31일이 없는 달도 있으므로 31일이 적은 것은 당연한 수치이지만 1일은 확실히 적은 수치인 것 같다.
- 구매일자는 주로 평일이 많고 주말에는 구매를 잘 하지 않는다는 것을 볼 수 있다
    - 부동산 매매업이 주로 평일에 영업을 해서 그런것으로 추정된다. 그렇다하더라도 수요일에는 구매가 적고 금요일에 구매가 많이 이루어진다
- 분기별로 비교했을 때, 2,3분기에 주로 구매한다

- 한 시기에 가격이 폭등한 적이 있는 것으로 보인다

In [17]:
# 시각화에 사용하고 이제 필요없어진 day_name 제거
train.drop('day_name', axis=1, inplace=True)
test.drop('day_name', axis=1, inplace=True)

## price : 타겟 변수인 집의 가격
- price는 타겟 변수이므로 y에 할당해준다

- price는 왼쪽으로 크게 치우쳐 있는 형태를 보인다.

따라서 y는 np.log1p() 함수를 통해 로그 변환을 해주고, 나중에 모델이 값을 예측한 후에 다시 np.expm1()을 활용해서 되돌려 줄 것이다. 

## bedrooms : 침실의 수, bathrooms : 침실당 화장실 개수

In [18]:
cols = ['bedrooms', 'bathrooms']

### 방의 개수와 화장실의 개수의 관계

- 여기서 최솟값인 0인 데이터가 눈에 띈다 방이 침실과 화장실이 없는 데이터는 무엇일까?
- 그리고 화장실에서 0.5 값은 무엇일까?

원본 데이터를 확인 해봤다
- bathrooms : Number of bathrooms, where .5 accounts for a room with a toilet but no shower
- 변기만 있고 shower 공간이 없는 화장실을 .5로 본다고 써있었다

In [19]:
train[(train.bedrooms == 0) | (train.bathrooms == 0)]

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day,day_of_week,quarter
4123,1415059200000000000,280000.0,1,0.00,600,24501,1.0,0,0,2,...,98045,47.5316,-121.749,990,22549,2014,11,4,1,4
6885,1419292800000000000,235000.0,0,0.00,1470,4800,2.0,0,0,3,...,98065,47.5265,-121.828,1060,7200,2014,12,23,1,4
7322,1410998400000000000,484000.0,1,0.00,690,23244,1.0,0,0,4,...,98053,47.6429,-121.955,1690,19290,2014,9,18,3,3
8826,1424217600000000000,320000.0,0,2.50,1490,7111,2.0,0,0,3,...,98065,47.5261,-121.826,1500,4675,2015,2,18,2,1
12781,1414540800000000000,265000.0,0,0.75,384,213444,1.0,0,0,3,...,98070,47.4177,-122.491,1920,224341,2014,10,29,2,4
13522,1411689600000000000,142000.0,0,0.00,290,20875,1.0,0,0,1,...,98024,47.5308,-121.888,1620,22850,2014,9,26,4,3


- 확실히 방이 0개인데 화장실이 2.5개인 집은 상당히 이상한 데이터로 보인다.
- 일단 6개이므로 제거해준다

In [20]:
odd_index = train[(train.bedrooms == 0) | (train.bathrooms == 0)].index

In [21]:
train.drop(odd_index, inplace=True)

- 0.75는 또 뭐일까 싶긴하지만 일단 여기까지만 하고 넘어가주자

### 평방 피트 관련 데이터
- sqft_living : 주거 공간의 평방 피트
- sqft_lot : 부지의 평방 피트
- sqft_above : 지하실을 제외한 평방 피트
- sqft_basement : 지하실의 평방 피트
- sqft_living15 : 2015년 기준 주거 공간의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)
- sqft_lot15 : 2015년 기준 부지의 평방 피트(집을 재건축했다면, 변화가 있을 수 있음)

In [22]:
sqft_df = train.loc[:,train.columns.str.contains('sqft')]

In [23]:
sqft_df.describe()

,sqft_living,sqft_lot,sqft_above,sqft_basement,sqft_living15,sqft_lot15
count,15029.000000,1.502900e+04,15029.000000,15029.000000,15029.000000,15029.000000
mean,2084.294497,1.528351e+04,1794.546810,289.747688,1992.966132,12808.982966
std,921.921821,4.259039e+04,831.620781,440.713233,691.481161,27687.680801
min,380.000000,5.200000e+02,380.000000,0.000000,399.000000,651.000000
25%,1430.000000,5.027000e+03,1190.000000,0.000000,1490.000000,5100.000000
50%,1920.000000,7.620000e+03,1570.000000,0.000000,1850.000000,7609.000000
75%,2560.000000,1.068800e+04,2230.000000,550.000000,2360.000000,10075.000000
max,13540.000000,1.651359e+06,9410.000000,4130.000000,6210.000000,871200.000000


- 15년 데이터는 sqft_living15, sqft_lot15, 14년 데이터는 sqft_living, sqft_lot을 써야하지 않을까?
- 지하실은 0인 데이터가 상당히 많아 보인다. 지하실이 있고 없고를 기준으로 파생변수를 만들어도 되지 않을까?

In [24]:
train.loc[:,train.columns.str.contains('sqft')] = np.log1p(train.loc[:,train.columns.str.contains('sqft')])

### 지하실이 있는집과 없는 집의 집값 차이는?

In [25]:
(train['sqft_basement'] > 0).value_counts()

sqft_basement
False    9137
True     5892
Name: count, dtype: int64

In [26]:
train['basement_yn'] = train['sqft_basement'] > 0

In [27]:
test['basement_yn'] = test['sqft_basement'] > 0

- 확실히 집 값이 비싼 집은 모두 지하실이 있는 것으로 보인다

### 15년에 재개발을 한 집은?

In [28]:
sqft_df.columns

Index(['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement',
       'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [29]:
def calc_diff(train):
    train['sqft_living_diff'] = train['sqft_living15'] - train['sqft_living']
    train['sqft_lot_diff'] = train['sqft_lot15'] - train['sqft_lot']

In [30]:
calc_diff(train)
calc_diff(test)

- 너무 많이 줄어든 것들이 이상하다

- 대다수의 값들이 부지크기가 변화한 것을 볼 수 있다
- 재개발을 하게 되면 집값이 상승할 수도 있지않을까? 하는 생각이 들었다.
- 이에 따라서 재개발에 대한 기대감으로 2014년에 계약했더라도 집값이 높게 측정될 수도 있다는 생각이 들어서 이 고민을 하게 되었다.
- 지금은 조금 복잡한 전처리가 될 것으로 보여지기에 추후 다시 생각해보기로 결정했다.

- 이번에는 2014년에 계약을 했다면 2014년 값을 쓰고 2015년에 계약했다면 2015년 값을 쓰는 데이터의 전처리만을 하고자한다.

### 2014년에 계약을 했다면 2014년 값을 쓰고 2015년에 계약했다면 2015년 값을 사용

In [31]:
def preprocessing_sqft(df):
    df['selected_sqft_living'] = np.where(df['year'] == 2014, df['sqft_living'], df['sqft_living15'])
    df['selected_sqft_lot'] = np.where(df['year'] == 2014, df['sqft_lot'], df['sqft_lot15'])
    df.drop(['sqft_living','sqft_living15','sqft_lot','sqft_lot15'], axis=1, inplace=True)
    # 합쳐준 이름을 'sqft_living','sqft_lot'으로 바꿔준다
    df.columns = df.columns.str.replace('selected_','')

In [32]:
preprocessing_sqft(train)
preprocessing_sqft(test)

In [33]:
# train.drop(['sqft_living','sqft_living15','sqft_lot','sqft_lot15'], axis=1, inplace=True)
train.head()

,date,price,bedrooms,bathrooms,floors,waterfront,view,condition,grade,sqft_above,...,year,month,day,day_of_week,quarter,basement_yn,sqft_living_diff,sqft_lot_diff,sqft_living,sqft_lot
0,1413158400000000000,221900.0,3,1.00,1.0,0,0,3,7,7.074117,...,2014,10,13,0,4,False,0.127054,0.000000,7.074117,8.639588
1,1424822400000000000,180000.0,2,1.00,1.0,0,0,3,6,6.647688,...,2015,2,25,2,1,False,1.261066,-0.215399,7.908755,8.995041
2,1424217600000000000,510000.0,3,2.00,1.0,0,0,3,8,7.427144,...,2015,2,18,2,1,False,0.068953,-0.074079,7.496097,8.923191
3,1403827200000000000,257500.0,3,2.25,2.0,0,0,3,7,7.447751,...,2014,6,27,4,2,False,0.266033,0.000000,7.447751,8.827615
4,1421280000000000000,291850.0,3,1.50,1.0,0,0,3,7,6.966967,...,2015,1,15,3,1,False,0.442169,0.000000,7.409136,9.181118


## 집 정보 관련 피처
- floors : 집의 층 수
- waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)

## 집 위치 정보 피처
- zipcode : 우편번호
- lat : 위도
- long : 경도

### 우편번호 별 집값 분포 시각화

- 확실히 zipcode는 위도와 경도(위치)에 따라 붙어있음을 알 수 있다

### 위도와 경도에 따른 집값 시각화

### 파생변수 생성
- zipcode별 집값의 중앙값을 파생변수로 생성한다
- 평균을 이용한다면 이상치의 영향을 받을 수도 있기에 중앙값을 이용하였다

In [34]:
median_price_by_zipcode = train.groupby('zipcode')['price'].median().reset_index()
median_price_by_zipcode.columns = ['zipcode', 'median_price']
train = train.merge(median_price_by_zipcode, on='zipcode')

In [35]:
median_price_by_zipcode.median_price.describe()

count    7.000000e+01
mean     5.049974e+05
std      2.609833e+05
min      2.350000e+05
25%      3.259375e+05
50%      4.506250e+05
75%      5.787500e+05
max      1.950000e+06
Name: median_price, dtype: float64

In [36]:
# test data에도 같은 처리를 해준다
test = test.merge(median_price_by_zipcode, on='zipcode')

In [37]:
train.head()

,date,price,bedrooms,bathrooms,floors,waterfront,view,condition,grade,sqft_above,...,month,day,day_of_week,quarter,basement_yn,sqft_living_diff,sqft_lot_diff,sqft_living,sqft_lot,median_price
0,1413158400000000000,221900.0,3,1.00,1.0,0,0,3,7,7.074117,...,10,13,0,4,False,0.127054,0.000000,7.074117,8.639588,273500.0
1,1403481600000000000,205425.0,2,1.00,1.0,0,0,4,6,6.781058,...,6,23,0,2,False,0.301491,0.000000,6.781058,8.821880,273500.0
2,1405555200000000000,445000.0,3,2.25,1.0,0,2,3,8,7.390799,...,7,17,3,3,True,0.236289,0.060438,7.650169,9.012133,273500.0
3,1424995200000000000,170000.0,2,1.00,1.0,0,0,3,6,6.758095,...,2,27,4,1,False,0.651042,0.510750,7.409136,9.079776,273500.0
4,1430438400000000000,245000.0,3,1.75,1.0,0,0,3,7,7.462215,...,5,1,4,2,False,-0.681157,1.915426,6.781058,11.269694,273500.0


### 연도 관련 피처
- yr_built : 집을 지은 년도
- yr_renovated : 집을 재건축한 년도

In [38]:
train[['yr_built', 'yr_renovated']].describe()

,yr_built,yr_renovated
count,15029.000000,15029.000000
mean,1971.098277,83.832391
std,29.409568,400.474919
min,1900.000000,0.000000
25%,1951.000000,0.000000
50%,1975.000000,0.000000
75%,1997.000000,0.000000
max,2015.000000,2015.000000


- 생각보다 오래된 건물들이 많이 있었다

- 재건축을 안한 집이 많아서 잘 보이지 않는다

0을 빼고 다시 시각화를 진행하였다

- 2014년에 재건축이 많았다

### 재건축 여부도 파생변수로 추가

In [39]:
train['renovated_yn'] = train['yr_renovated'] > 0
test['renovated_yn'] = test['yr_renovated'] > 0

### 파생변수 생성
**집 나이 계산**:
   - `np.where` 함수를 사용하여 조건을 설정하여 집 나이를 계산한다.
     - 만약 `yr_renovated` 값이 `0`이 아니면, 즉 재건축이 되었으면 `year - yr_renovated`를 계산한다.
     - 그렇지 않으면 `year - yr_built`를 계산한다.
   - 이를 통해 `house_age`라는 새로운 컬럼을 생성

In [40]:
def calc_age(df):
    df['house_age'] = np.where(df['yr_renovated'] != 0, df['year'] - df['yr_renovated'], df['year'] - df['yr_built'])

In [41]:
calc_age(train)

In [42]:
calc_age(test)

### 집에 대한 평가 피처
- view : 집이 얼마나 좋아 보이는지의 정도
- condition : 집의 전반적인 상태
- grade : King County grading 시스템 기준으로 매긴 집의 등급

- 확실히 집 상태의 평가가 높아지면 더 값이 비싸지는 경향을 볼 수 있다

### 파생변수 생성
- view, condition, grade의 평균을 파생변수로 생성하고자 한다

In [43]:
def get_score(df):
    df['score'] = df[['view', 'condition', 'grade']].mean(axis=1)

In [44]:
get_score(train)

In [45]:
get_score(test)

## 피처 선정

In [46]:
train['price'] = np.log1p(train['price'])

In [47]:
correlation_matrix = train.corr()
correlation_matrix.price

date                0.001475
price               1.000000
bedrooms            0.359418
bathrooms           0.551879
floors              0.317828
waterfront          0.172599
view                0.347493
condition           0.042161
grade               0.707611
sqft_above          0.591088
sqft_basement       0.231755
yr_built            0.076438
yr_renovated        0.127487
zipcode            -0.039441
lat                 0.444423
long                0.055219
year                0.009736
month              -0.013622
day                -0.020304
day_of_week        -0.000471
quarter            -0.011881
basement_yn         0.207486
sqft_living_diff   -0.310703
sqft_lot_diff      -0.075450
sqft_living         0.657869
sqft_lot            0.140776
median_price        0.698083
renovated_yn        0.127196
house_age          -0.128667
score               0.690882
Name: price, dtype: float64

In [48]:
# drop_cols = correlation_matrix.columns[np.abs(correlation_matrix.price) < 0.1]
# train.drop(drop_cols, axis=1, inplace=True)

In [49]:
train.columns

Index(['date', 'price', 'bedrooms', 'bathrooms', 'floors', 'waterfront',
       'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'year', 'month', 'day',
       'day_of_week', 'quarter', 'basement_yn', 'sqft_living_diff',
       'sqft_lot_diff', 'sqft_living', 'sqft_lot', 'median_price',
       'renovated_yn', 'house_age', 'score'],
      dtype='object')

In [50]:
# test.drop(drop_cols, axis=1, inplace=True)

In [51]:
for df in [train, test]:
    
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement'] + df['sqft_lot']
    
    # 15년도가 아닌 주변 15개 가구 평균값
    df['sqft_total_size'] = df['sqft_living'] + df['sqft_lot']
    
    # 재건축 여부 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')

## zipcode를 어떻게 사용해볼까 고민하다가 아래의 코드를 발견해서 참고하였다
- https://www.kaggle.com/code/reddust/feat-groupbyzipcode

In [52]:
for data in [train, test]:
    data['zipcode'] = data['zipcode'].astype(str)
    # 45, 5, 35, 4
    data['zipcode-3'] = 'z_' + data['zipcode'].str[2:3]
    data['zipcode-4'] = 'z_' + data['zipcode'].str[3:4]
    data['zipcode-5'] = 'z_' + data['zipcode'].str[4:5]
    data['zipcode-34'] = 'z_' + data['zipcode'].str[2:4]
    data['zipcode-45'] = 'z_' + data['zipcode'].str[3:5]
    data['zipcode-35'] = 'z_' + data['zipcode'].str[2:3] + data['zipcode'].str[4:5]
    
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# zipcode LabelEncoding
for df in [train, test]:
    le = LabelEncoder()
    df['zipcode'] = le.fit_transform(df['zipcode'])
    df['zipcode-3'] = le.fit_transform(df['zipcode-3'])
    df['zipcode-4'] = le.fit_transform(df['zipcode-4'])
    df['zipcode-5'] = le.fit_transform(df['zipcode-5'])
    df['zipcode-34'] = le.fit_transform(df['zipcode-34'])
    df['zipcode-45'] = le.fit_transform(df['zipcode-45'])
    df['zipcode-35'] = le.fit_transform(df['zipcode-35'])

In [53]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

for df in [train, test]:
    coord = df[['lat','long']]
    pca = PCA(n_components=2)
    pca.fit(coord)

    coord_pca = pca.transform(coord)

    df['coord_pca1'] = coord_pca[:, 0]
    df['coord_pca2'] = coord_pca[:, 1]

In [54]:
train['per_price'] = train['price']/train['sqft_total_size']
zipcode_price = train.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()
train = pd.merge(train, zipcode_price, how='left',on='zipcode')
test = pd.merge(test, zipcode_price, how='left',on='zipcode')

for df in [train, test]:
    df['zipcode_mean'] = df['mean'] * df['sqft_total_size']
    df['zipcode_var'] = df['var'] * df['sqft_total_size']
    del df['mean']; del df['var']

In [55]:
train.head()

,date,price,bedrooms,bathrooms,floors,waterfront,view,condition,grade,sqft_above,...,zipcode-4,zipcode-5,zipcode-34,zipcode-45,zipcode-35,coord_pca1,coord_pca2,per_price,zipcode_mean,zipcode_var
0,1413158400000000000,12.309987,3,1.00,1.0,0,0,3,7,7.074117,...,7,8,16,54,16,-0.000699,-0.065735,0.783392,12.114741,0.008446
1,1403481600000000000,12.232841,2,1.00,1.0,0,0,4,6,6.781058,...,7,8,16,54,16,0.012834,-0.067722,0.784009,12.029344,0.008386
2,1405555200000000000,13.005832,3,2.25,1.0,0,2,3,8,7.390799,...,7,8,16,54,16,0.010551,-0.058889,0.780554,12.846078,0.008955
3,1424995200000000000,12.043560,2,1.00,1.0,0,0,3,6,6.758095,...,7,8,16,54,16,-0.007949,-0.080343,0.730404,12.712401,0.008862
4,1430438400000000000,12.409018,3,1.75,1.0,0,0,3,7,7.462215,...,7,8,16,54,16,0.038283,-0.054405,0.687452,13.916527,0.009702


In [56]:
train.shape

(15029, 43)

## 모델링

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [58]:
# 데이터 분할 (Train, Validation, Test)
y = train['price']
y = np.log1p(y)

X = train.drop(['price','per_price'], axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
# 테스트 데이터
X_test = test

# # 표준화
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)

## 모델 도입 및 평가

### 모델 및 평가 지표 선정 이유
- 선택한 모델: 앙상블 모델 (랜덤 포레스트, 그래디언트 부스팅, 배깅)
    - 선정 이유: 앙상블 모델은 여러 모델의 예측을 결합하여 예측 성능을 향상시키며, 단일 모델보다 일반화 성능이 뛰어나다.

- 평가 지표: RMSE(Root Mean Squared Error)
    - 선정 이유: RMSE는 예측 값과 실제 값 간의 차이를 제곱한 후 평균을 구해 루트를 취한다. 단위가 타깃 변수의 단위와 같아 해석이 용이하고, 큰 오차에 더 큰 패널티를 부여한다.

In [59]:
gboost = GradientBoostingRegressor(random_state=2019)
xgboost = xgb.XGBRegressor(random_state=2019)
# lightgbm = lgb.LGBMRegressor(random_state=2019)

models = [{'model':gboost, 'name':'GradientBoosting'}, {'model':xgboost, 'name':'XGBoost'}]

In [60]:
def get_cv_score(models):
    kfold = KFold(n_splits=5, shuffle=True).get_n_splits(X.values)
    for m in models:
        print("Model {} CV score : {:.4f}".format(m['name'], np.mean(cross_val_score(m['model'], X.values, y)), 
                                             kf=kfold))

In [61]:
get_cv_score(models)

Model GradientBoosting CV score : 0.8555
Model XGBoost CV score : 0.8584


### Make Submission

회귀 모델의 경우에는 cross_val_score 함수가 R<sup>2</sup>를 반환합니다.<br>
R<sup>2</sup> 값이 1에 가까울수록 모델이 데이터를 잘 표현함을 나타냅니다. 3개 트리 모델이 상당히 훈련 데이터에 대해 괜찮은 성능을 보여주고 있습니다.<br> 훈련 데이터셋으로 3개 모델을 학습시키고, Average Blending을 통해 제출 결과를 만들겠습니다.,

In [62]:
def AveragingBlending(models, X, y, sub_x):
    for m in models : 
        m['model'].fit(X.values, y)
    
    predictions = np.column_stack([
        m['model'].predict(sub_x.values) for m in models
    ])
    return np.mean(predictions, axis=1)

In [63]:
y_pred = AveragingBlending(models, X, y, test)

In [65]:
submission_csv_path = './data/housing/sample_submission.csv'
submission = pd.read_csv(submission_csv_path)

In [66]:
submission['price'] = y_pred
submission.to_csv('submission_blend2.csv', index=False)

## 

In [ ]:
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test), np.expm1(y_pred)))

In [ ]:
def get_scores(models, X, y):
    df = {}

    for model in models:
    # 모델 이름 획득
        model_name = model.__class__.__name__

        # train, test 데이터셋 분리
        # random_state를 사용하여 고정하고 train과 test 셋의 비율은 8:2로 합니다.
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_state, test_size=0.2)

        # 모델 학습
        model.fit(X_train, y_train)
        
        # 예측
        y_pred = model.predict(X_test)

        # 예측 결과의 rmse값 저장
        df[model_name] = rmse(y_test, y_pred)
        
        # data frame에 저장
        score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)

    return score_df

get_scores(models, X, y)

## 하이퍼 파라미터 튜닝의 최강자, 그리드 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
"""
다음과 같은 과정을 진행할 수 있는 `my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5)` 함수를 구현해 보세요.

1. GridSearchCV 모델로 `model`을 초기화합니다.
2. 모델을 fitting 합니다.
3. params, score에 각 조합에 대한 결과를 저장합니다. 
4. 데이터 프레임을 생성하고, RMSLE 값을 추가한 후 점수가 높은 순서로 정렬한 `results`를 반환합니다.
"""

In [ ]:
def my_GridSearch(model, train, y, param_grid, verbose=2, n_jobs=5):
    # GridSearchCV 모델로 초기화
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', \
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    
    # 모델 fitting
    grid_model.fit(train, y)

    # 결과값 저장
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    # 데이터 프레임 생성
    results = pd.DataFrame(params)
    results['score'] = score
    
    # RMSLE 값 계산 후 정렬
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')

    return results

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [1, 10],
}

In [ ]:

model = lgb.LGBMRegressor(random_state=random_state)
my_GridSearch(model, X, y, param_grid, verbose=2, n_jobs=5)

model = lgb.LGBMRegressor(max_depth=10, n_estimators=100, random_state=random_state)
model.fit(X, y)
prediction = model.predict(X_test)
prediction = np.expm1(prediction)
prediction

data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'

submission_path = join(data_dir, 'sample_submission.csv')
submission = pd.read_csv(submission_path)
submission.head()

submission['price'] = prediction
submission.head()

submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, 'lgbm', '0.164399')
submission.to_csv(submission_csv_path, index=False)
print(submission_csv_path)

"""
아래의 과정을 수행하는 `save_submission(model, train, y, test, model_name, rmsle)` 함수를 구현해 주세요.
1. 모델을 `train`, `y`로 학습시킵니다.
2. `test`에 대해 예측합니다.
3. 예측값을 `np.expm1`으로 변환하고, `submission_model_name_RMSLE_100000.csv` 형태의 `csv` 파일을 저장합니다.
"""

def save_submission(model, train, y, test, model_name, rmsle=None):
    model.fit(train, y)
    prediction = model.predict(test)
    prediction = np.expm1(prediction)
    data_dir = os.getenv('HOME')+'/aiffel/kaggle_kakr_housing/data'
    submission_path = join(data_dir, 'sample_submission.csv')
    submission = pd.read_csv(submission_path)
    submission['price'] = prediction
    submission_csv_path = '{}/submission_{}_RMSLE_{}.csv'.format(data_dir, model_name, rmsle)
    submission.to_csv(submission_csv_path, index=False)
    print('{} saved!'.format(submission_csv_path))


In [ ]:
models

### 최종 결과

- 데이터 분석을 하다가 시간을 많이 써서 좋은 결과를 내는데 생각보다 시간을 많이 못써서 아쉽다.
- 다음에는 시간분배를 좀 더 잘 해봐야겠다